In [1]:
import pandas as pd
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

In [26]:
q = "select org, count(id), sum(amount) from transactions where recurring=0 and status='A' and date='{}' group by org"

df_2022 = redshift_query_read(q.format('2022-11-29'), schema='production')
df_2021 = redshift_query_read(q.format('2021-11-30'), schema='production')

In [27]:
print("{:,} orgs 2021".format(len(df_2021)))
print("{:,} orgs 2022".format(len(df_2022)))

1,599 orgs 2021
1,628 orgs 2022


In [28]:
orgs_lost = df_2021[~df_2021['org'].isin(df_2022['org'].tolist())]['org'].tolist()

In [31]:
q = "select org, count(id), sum(amount), max(date) from transactions where recurring=0 and status='A' and date_part('year', date)=2022 group by org"
orgs_2022 = redshift_query_read(q, schema='production')

In [32]:
orgs_2022_lost = orgs_2022[orgs_2022['org'].isin(orgs_lost)]
orgs_2022_lost_still_active = orgs_2022_lost[orgs_2022_lost['max'].dt.month>=11]

print("{:,} orgs lost from GT 2021 to 2022".format(len(orgs_lost)))
print("{:,} orgs lost that were active in 2022".format(len(orgs_2022_lost)))
print("{:,} orgs lost that were active in 11/2022".format(len(orgs_2022_lost_still_active)))

481 orgs lost from GT 2021 to 2022
435 orgs lost that were active in 2022
268 orgs lost that were active in 11/2022
